<a href="https://colab.research.google.com/github/yurekan/Cropance/blob/notebook/Crop_Yield_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import keras
import os
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.models import Sequential
from keras.layers import InputLayer, Dense, Dropout, Activation
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Crop_recommendation.csv')
data.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   N            2200 non-null   int64  
 1   P            2200 non-null   int64  
 2   K            2200 non-null   int64  
 3   temperature  2200 non-null   float64
 4   humidity     2200 non-null   float64
 5   ph           2200 non-null   float64
 6   rainfall     2200 non-null   float64
 7   label        2200 non-null   object 
dtypes: float64(4), int64(3), object(1)
memory usage: 137.6+ KB


In [ ]:
data.describe()

,N,P,K,temperature,humidity,ph,rainfall
count,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000
mean,50.551818,53.362727,48.149091,25.616244,71.481779,6.469480,103.463655
std,36.917334,32.985883,50.647931,5.063749,22.263812,0.773938,54.958389
min,0.000000,5.000000,5.000000,8.825675,14.258040,3.504752,20.211267
25%,21.000000,28.000000,20.000000,22.769375,60.261953,5.971693,64.551686
50%,37.000000,51.000000,32.000000,25.598693,80.473146,6.425045,94.867624
75%,84.250000,68.000000,49.000000,28.561654,89.948771,6.923643,124.267508
max,140.000000,145.000000,205.000000,43.675493,99.981876,9.935091,298.560117


In [ ]:
target = data['label'].unique()
target

array(['rice', 'maize', 'chickpea', 'kidneybeans', 'pigeonpeas',
       'mothbeans', 'mungbean', 'blackgram', 'lentil', 'pomegranate',
       'banana', 'mango', 'grapes', 'watermelon', 'muskmelon', 'apple',
       'orange', 'papaya', 'coconut', 'cotton', 'jute', 'coffee'],
      dtype=object)

In [ ]:
le = LabelEncoder()
le.fit(data.label)
data['label'] = le.transform(data.label)

In [ ]:
data['label'].unique()

array([20, 11,  3,  9, 18, 13, 14,  2, 10, 19,  1, 12,  7, 21, 15,  0, 16,
       17,  4,  6,  8,  5])

In [ ]:
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
label_mapping

{'apple': 0,
 'banana': 1,
 'blackgram': 2,
 'chickpea': 3,
 'coconut': 4,
 'coffee': 5,
 'cotton': 6,
 'grapes': 7,
 'jute': 8,
 'kidneybeans': 9,
 'lentil': 10,
 'maize': 11,
 'mango': 12,
 'mothbeans': 13,
 'mungbean': 14,
 'muskmelon': 15,
 'orange': 16,
 'papaya': 17,
 'pigeonpeas': 18,
 'pomegranate': 19,
 'rice': 20,
 'watermelon': 21}

In [ ]:
X_data = data.iloc[:,0:7].to_numpy()
y_data = data['label'].to_numpy()

In [ ]:
y_data_temp = np.zeros((len(y_data), len(label_mapping)))
for i in range(len(y_data)):
  y_data_temp[i,y_data[i]] = 1

In [ ]:
y_data_temp

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data_temp, test_size=0.3, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [ ]:
model = Sequential()

model.add(Dense(64, input_dim=len(X_train[0,:]), activation='tanh'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(22, activation='softmax'))

In [ ]:
model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=[
        tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                512       
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 128)               16512     
                                                                 
 dense_4 (Dense)             (None, 22)                2838      
                                                                 
Total params: 44,694
Trainable params: 44,694
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train, epochs=50)

Epoch 1/50
49/49 [==============================] - 1s 4ms/step - loss: 1.9537 - accuracy: 0.5084 - precision: 0.9509 - recall: 0.1383
Epoch 2/50
49/49 [==============================] - 0s 4ms/step - loss: 0.5945 - accuracy: 0.8052 - precision: 0.9136 - recall: 0.6935
Epoch 3/50
49/49 [==============================] - 0s 4ms/step - loss: 0.3723 - accuracy: 0.8818 - precision: 0.9188 - recall: 0.8299
Epoch 4/50
49/49 [==============================] - 0s 4ms/step - loss: 0.3121 - accuracy: 0.8948 - precision: 0.9173 - recall: 0.8721
Epoch 5/50
49/49 [==============================] - 0s 4ms/step - loss: 0.2354 - accuracy: 0.9162 - precision: 0.9317 - recall: 0.9039
Epoch 6/50
49/49 [==============================] - 0s 4ms/step - loss: 0.2269 - accuracy: 0.9221 - precision: 0.9354 - recall: 0.9032
Epoch 7/50
49/49 [==============================] - 0s 4ms/step - loss: 0.1749 - accuracy: 0.9383 - precision: 0.9439 - recall: 0.9292
Epoch 8/50
49/49 [==============================] - 0s 

In [ ]:
!pip install h5py pyyaml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
model.fit(X_train, y_train, epochs=50, callbacks=[cp_callback])

Epoch 1/50
47/49 [===========================>..] - ETA: 0s - loss: 0.0772 - accuracy: 0.9674 - precision: 0.9674 - recall: 0.9674
Epoch 1: saving model to training_1/cp.ckpt
49/49 [==============================] - 0s 10ms/step - loss: 0.0761 - accuracy: 0.9682 - precision: 0.9682 - recall: 0.9682
Epoch 2/50
49/49 [==============================] - ETA: 0s - loss: 0.0788 - accuracy: 0.9695 - precision: 0.9695 - recall: 0.9695
Epoch 2: saving model to training_1/cp.ckpt
49/49 [==============================] - 0s 5ms/step - loss: 0.0788 - accuracy: 0.9695 - precision: 0.9695 - recall: 0.9695
Epoch 3/50
37/49 [=====================>........] - ETA: 0s - loss: 0.0906 - accuracy: 0.9721 - precision: 0.9746 - recall: 0.9721
Epoch 3: saving model to training_1/cp.ckpt
49/49 [==============================] - 0s 5ms/step - loss: 0.0929 - accuracy: 0.9708 - precision: 0.9727 - recall: 0.9701
Epoch 4/50
47/49 [===========================>..] - ETA: 0s - loss: 0.0667 - accuracy: 0.9707 - precis

In [ ]:
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [ ]:
score = model.evaluate(X_test, y_test, batch_size=128)

6/6 [==============================] - 1s 4ms/step - loss: 0.1556 - accuracy: 0.9515 - precision: 0.9528 - recall: 0.9485


In [ ]:
model.metrics_names

['loss', 'accuracy', 'precision', 'recall']

In [ ]:
score

[0.15561270713806152, 0.9515151381492615, 0.95281583070755, 0.9484848380088806]

In [ ]:
y_pred = model.predict(X_test)
y_pred

21/21 [==============================] - 0s 2ms/step


array([[1.7848370e-10, 6.1094088e-14, 9.3276584e-12, ..., 1.5381261e-11,
        2.4209465e-12, 6.7335640e-07],
       [4.4990656e-09, 1.0073225e-08, 8.2380298e-15, ..., 1.5609984e-07,
        1.9910946e-12, 9.9999660e-01],
       [9.1303818e-06, 2.7494702e-08, 1.8589284e-10, ..., 4.9502319e-06,
        1.9417597e-10, 3.5347441e-07],
       ...,
       [5.2446367e-06, 2.3476703e-08, 3.0146188e-11, ..., 1.8284156e-05,
        1.1814838e-10, 3.5053120e-06],
       [3.3940123e-06, 9.4356773e-07, 7.8075853e-12, ..., 8.5951989e-11,
        5.0242899e-13, 3.4041747e-08],
       [5.0101518e-09, 2.1778970e-08, 1.5660577e-13, ..., 4.8628648e-09,
        1.1153761e-11, 9.9998629e-01]], dtype=float32)

In [ ]:
predicted = np.argmax(y_pred, axis=1)

In [ ]:
def get_crop_name(val):
  for key, value in label_mapping.items():
    if value == val:
      return key
print(get_crop_name(predicted[0]))

muskmelon
